# A "MuSQL" Exhibition

Here are a few SQL queries of an online music store's database.

**Loading SQL and the importing the database:**

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

**Viewing a table of all the tables in the database:**

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


**Viewing the first 10 rows of the customer table:**

In [3]:
%%sql
SELECT *
  FROM customer
 LIMIT 10;

 * sqlite:///chinook.db
Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4
6,Helena,Holý,None,Rilská 3174/6,Prague,None,Czech Republic,14300,+420 2 4177 0449,None,hholy@gmail.com,5
7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
9,Kara,Nielsen,None,Sønder Boulevard 51,Copenhagen,None,Denmark,1720,+453 3331 9991,None,kara.nielsen@jubii.dk,4
10,Eduardo,Martins,Woodstock Discos,"Rua Dr. Falcão Filho, 155",São Paulo,SP,Brazil,01007-010,+55 (11) 3033-5446,+55 (11) 3033-4564,eduardo@woodstock.com.br,4


**Creating a frequency table of genres:**

In [4]:
%%sql
WITH track_genre AS
    (
    SELECT t.name track_name,
           g.name genre,
           i.billing_country country
      FROM track t
      LEFT JOIN genre g ON g.genre_id = t.genre_id
      LEFT JOIN invoice_line il ON il.track_id = t.track_id
      LEFT JOIN invoice i ON i.invoice_id = il.invoice_id
     WHERE country = 'USA'
    ),
genre_count AS
    (
    SELECT genre, 
           COUNT(track_name) tracks
      FROM track_genre
     GROUP BY genre
     ORDER BY tracks DESC
    )
SELECT *,
       ROUND(CAST(tracks AS Float)/(
               SELECT SUM(tracks)
                 FROM genre_count)*100,2) percentage
  FROM genre_count

 * sqlite:///chinook.db
Done.


genre,tracks,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


- Rock is by far the most popular genre, comprising more than 53 percent of all tracks.
- Alternative & Punk and Metal, both subdenominatons of rock, are the next most popular at 12.4 and 11.8 percent respectively.
- Based on this information, it appears that rock and rock-adjacent music does the best among our customer base. The company would be wise to continue offering tracks of these genres.

**Finding total sales and customer count by employee:**

In [5]:
%%sql
WITH invoice_employee AS
(
SELECT e.first_name || ' ' || e.last_name employee_name,
       i.total total,
       c.customer_id
  FROM invoice i
  LEFT JOIN customer c ON c.customer_id = i.customer_id
  LEFT JOIN employee e ON c.support_rep_id = e.employee_id
)

SELECT employee_name,
       round(SUM(total), 2) total,
       COUNT(DISTINCT customer_id) num_customers,
       round(round(SUM(total), 2)/COUNT(DISTINCT customer_id), 2) avg_value_per_customer
  FROM invoice_employee
 GROUP BY employee_name;

 * sqlite:///chinook.db
Done.


employee_name,total,num_customers,avg_value_per_customer
Jane Peacock,1731.51,21,82.45
Margaret Park,1584.0,20,79.2
Steve Johnson,1393.92,18,77.44


- Jane Peacock had the highest sales and total customers. Despite having only 1 less customer, Margaret Park brought in nearly $150 less than Peacock.
- Peacock also had the highest average value per customer.
- Steve Johnson scored the lowest across all 3 metrics. Poor Steve.

**Finding total customers, total sales, avg customer value and avg order value by country (grouping countries with only 1 customer as "Other"):**

In [6]:
%%sql
WITH customer_total_by_country AS
(
SELECT 
       CASE
           WHEN (
                 SELECT count(*)
                   FROM customer
                  WHERE country = c.country
                ) = 1 THEN 'Other'
                ELSE c.country
            END AS country,
            cu.customer_id,
            i.total
  FROM invoice i
  LEFT JOIN customer cu ON cu.customer_id = i.customer_id
  LEFT JOIN customer c ON c.customer_id = cu.customer_id
),

sales_and_averages AS
(
SELECT  country,
        COUNT(DISTINCT customer_id) customers,
        ROUND(SUM(total),2) total_sales,
        ROUND(SUM(total)/COUNT(DISTINCT customer_id),2) avg_customer_value,
        ROUND(AVG(total),2) avg_order,
        CASE
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
  FROM customer_total_by_country
 GROUP BY country
 ORDER BY sort ASC, total_sales DESC
)

SELECT country, customers, total_sales, avg_customer_value, avg_order
  FROM sales_and_averages;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,avg_customer_value,avg_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


- The US has the most customers and highest total sales. 
- Brazil had the highest average customer value among the top 5 countries by sales.
- The Czech Republic only has 2 customers, but has the highest average customer value and average order value.
- The Czech Republic, Portugal, and India only had 2 customers each, but all recorded the highest average customer values.

### Conclusion

Our cursory analysis yielded a few salient observations:

1. Rock and rock-adjacent (punk, alternative) songs are the most popular.
2. Jane Peacock is the highest performing salesperson. Steve might need more training.
3. While the US had the highest total sales, several countries with only a couple customers each recorded the highest average customer values. In other words, on average, customers in these smaller European and Asian countries spend more. These could be potential growth areas in the future.